In [1]:
import pandas as pd
from pyhanlp import *
import jieba
import numpy as np

df = pd.read_excel('Lawsnote法律黑客松＿判決摘要.xlsx',sheet_name='慰撫金')

# 濾除雜質
for idx, row in df.iterrows():
    for col in df.columns:
        if type(df.iloc[idx][col]) == str:
            value = df.iloc[idx][col]
            value = value.strip()
            value = value.replace('未滿','').replace('不明','').replace('未主張','')
            value = value.replace('目前沒有工作','').replace('目前在監服刑','').replace('在監執行','')
            value = value.replace('無','').replace('目前在監服刑','').replace('在監執行','')
            df.iloc[idx][col] = value 
            
        if col == '年齡':
            if ('nan' in value) or (value == ''): df.iloc[idx][col] = 0
                    
                
                                        
# 留下必要row           
col = [c for c in df.columns if c not in ['法院']]
df = df.drop([0,1])
df = df.reset_index(drop=True)
df = df.dropna(subset=['最後判賠金額'])
df = df[df["犯罪類型"] != 'X']

In [2]:
df.head()

,法院,主文,犯罪類型,職業,產業,階級,學歷,年齡,等級,數額,財產總額,財產種類,原告主張,原告年紀,被告主張,有利原告,有利被告,原告要求金額,最後判賠金額
0,臺灣新北地方法院107年訴字第1661號民事判決,臺灣新北地方法院民事判決107年度訴字第1661號原告0000-000000（真實姓名年籍均...,強制性交,"餐飲及溫泉會館,老闆",住宿及餐飲業,老闆,NaN,NaN,經濟狀況良好,1000000,NaN,NaN,服用藥物昏沈不能抗拒,52,,趁原告服用安眠藥法抗拒\n事後悔意從未道歉\n指稱原告設局引誘被告,NaN,800000,500000
2,臺灣高等法院107年上易字第56號民事判決,臺灣高等法院民事判決107年度上易字第56號上訴人王凱民訴訟代理人王俊傑律師被上訴人A女(0...,強制性交未遂,待業,NaN,NaN,大學肄業,NaN,NaN,52540,NaN,NaN,因酒醉不知不能抗拒,NaN,當日因飲酒過量精神恍惚\n證人之證詞互有矛盾\nDNA未為23種型別比對\n請求之賠償金額過高,被告利用原告酒醉不知不能抗拒,NaN,1300000,900000
3,臺灣新北地方法院107年訴字第727號民事判決,臺灣新北地方法院民事判決107年度訴字第727號原告林明子訴訟代理人蔣宗翰律師張君維被告徐松...,性騷擾,待業,NaN,NaN,國小畢業,NaN,NaN,36000,NaN,"投資:1,不動產:1",被告不滿原告以徒手拍打其臀部，而徒手反推原告\n原告當場倒地，造成原告背部外傷及骨折,NaN,正當防衛，傷害故意\n被告才是受害人,NaN,NaN,700000,50000
4,臺灣臺北地方法院106年醫字第16號民事判決,臺灣臺北地方法院民事判決106年度醫字第16號原告A女（真實姓名、住居所詳卷）訴訟代理人周珊...,性騷擾,醫師,醫療保健業,員工,NaN,NaN,NaN,1914876,NaN,NaN,被告趁原告尋求醫療協助之際，要求發生性行為\n醫療行為不當，致原告腿部神經病變,NaN,術前已盡評估之事\n原告為護士，於術前應已了解風險,趁病人不適且法逃脫醫師治療之依賴\n違反道德,NaN,300000,250000
5,臺灣高等法院臺中分院107年訴易字第78號民事判決,臺灣高等法院臺中分院民事判決107年度訴易字第78號原告A女（代號0000-000000）（...,強制性交,待業,NaN,NaN,專科肄業,NaN,輕度精神障礙\n存款\n領有重大傷病證明卡,0,NaN,NaN,被告明知其為低智商而強制性交,NaN,二人互有情愫\n並未暴力脅迫原告\n經濟能力不佳，不應請求高額求償金,被告明知其為低智商而強制性交,NaN,1000000,400000


# Preprocessing

In [3]:
def education(text):
    if text!= None and type(text) == str:
        try:
            text = text.replace('\n','')
            seg_list = jieba.cut(text)
            seg_list = [s for s in seg_list]

            seg_list_1 = "".join(seg_list[:-1])
            seg_list_2 = seg_list[-1]

            if 'nan' in seg_list_2: return [0,0]
            if seg_list_1 == '': return [seg_list_2]
            return [seg_list_1,seg_list_2]
        except Exception as e:
            return [0,0]
    else: return [0,0]

def study(v):
    level = 0
    if type(v[0]) == str:
        text = v[0]
        if text in ['國小']: level = 0
        elif text in ['初中','國中','國 中']: level =  1
        elif text in ['高中','高職','高工','家商']: level =  2
        elif text in ['大學','空中大學','二專','專科','大專','技術學院']: level =  3
        elif text in ['研究','碩士']: level =  4
        elif text in ['博士']: level =  5
        if len(v)>1:
            if '肄業' in v[1]: return (level-0.5)
            else: return level

    
def extract_age(text):
    if type(text) == str:
        seg_list = jieba.cut(text)
        for s in seg_list:
            if s.isdigit(): return int(s)
    else:
        return text   
    

    
    

# 年齡處理
df['年齡'] = df['年齡'].fillna(0)
df['年齡'] = df['年齡'].apply(extract_age)
age_count = [v for v in df['年齡'] if v > 0]
df['年齡'] = df['年齡'].apply(lambda x : x if x>0 else np.mean(age_count)) # 年齡填充


# 學歷處理  
df['學歷'] = df['學歷'].apply(education)
level_dict = [v[0] for v in df['學歷'] if type(v[0]) == str]
level_dict = ['國小','初中','國中','國 中','高中','高職','高工','大學','空中大學','二專','碩士']           
df['學歷'] = df['學歷'].apply(study)  
study_count = [v for v in df['學歷'] if v > 0]
df['學歷'] = df['學歷'].apply(lambda x : x if x>0 else np.mean(study_count)) # 學歷填充

# 原告年紀處理 
df['原告年紀'] = df['原告年紀'].fillna(0)
hurt_age_count = [int(v) for v in df['原告年紀'] if int(v) > 0]
df['原告年紀'] = df['原告年紀'].apply(lambda x : x if x>0 else np.max(hurt_age_count)) # 原告年紀

# df
df[['職業','產業','階級','學歷','年齡']].tail()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\jack\AppData\Local\Temp\jieba.cache
Loading model cost 0.896 seconds.
Prefix dict has been built succesfully.


,職業,產業,階級,學歷,年齡
221,",公車司機",公共汽車客運業,員工,1.954955,36.272727
222,",法師",其他個人服務業,老闆,2.000000,36.272727
223,待業,NaN,NaN,1.954955,18.000000
224,",戲劇製作人","創作及藝術表演業, 藝文服務業",主管,1.954955,36.272727
225,待業,NaN,NaN,1.954955,36.272727


# 薪情平臺匯出資料_數額引用

In [4]:
# 數額填充
import operator
employee = pd.read_excel('Lawsnote法律黑客松＿判決摘要.xlsx',sheet_name='薪情平臺匯出資料') # 薪情平臺匯出資料

employee.head()  
employee_money ={} # 薪資排行榜
employee_mapping={} 
for idx, row in employee.iterrows():
    employee_money[list(row)[0]] = list(row)[2]    
    if type(list(row)[1]) == str:
        employee_mapping[list(row)[0]] = list(row)[1].split(',')
    

employee_rank = sorted(employee_money.items(), key=operator.itemgetter(1),reverse=True)
employee_rank = [e[0] for e in employee_rank]          

def fill_money(row):
    if row['數額']: return row['數額']
    else:
        return employee_money[row['產業']]

    
    
# df['數額'] = df.apply(fill_money)
df['階級'] = df['階級'].fillna('待業')
df['產業'] = df['產業'].fillna('待業')
# df['財產種類'] = df['財產種類'].fillna('無財產')
df[['職業','產業','階級','學歷','年齡','數額']].head()

,職業,產業,階級,學歷,年齡,數額
0,"餐飲及溫泉會館,老闆",住宿及餐飲業,老闆,1.954955,36.272727,1000000
2,待業,待業,待業,2.500000,36.272727,52540
3,待業,待業,待業,1.954955,36.272727,36000
4,醫師,醫療保健業,員工,1.954955,36.272727,1914876
5,待業,待業,待業,2.500000,36.272727,0


# 把犯罪類型、階級、財產種類做one hot encoding

In [5]:
# 刪除不必要的columns
df = df.drop(["法院", "主文", "職業", "產業", "等級", "財產總額"], axis=1)
# 所有\n化為，
df.replace('\n','，', inplace=True, regex=True)

# 刪除犯罪類型為nan的index
Nanlist = df[df["犯罪類型"].isnull().values==True].index.tolist()
df = df.drop(Nanlist)

df.reset_index(drop=True, inplace=True)
df.head()

,犯罪類型,階級,學歷,年齡,數額,財產種類,原告主張,原告年紀,被告主張,有利原告,有利被告,原告要求金額,最後判賠金額
0,強制性交,老闆,1.954955,36.272727,1000000,NaN,服用藥物昏沈不能抗拒,52,,趁原告服用安眠藥法抗拒，事後悔意從未道歉，指稱原告設局引誘被告,NaN,800000,500000
1,強制性交未遂,待業,2.500000,36.272727,52540,NaN,因酒醉不知不能抗拒,52,當日因飲酒過量精神恍惚，證人之證詞互有矛盾，DNA未為23種型別比對，請求之賠償金額過高,被告利用原告酒醉不知不能抗拒,NaN,1300000,900000
2,性騷擾,待業,1.954955,36.272727,36000,"投資:1,不動產:1",被告不滿原告以徒手拍打其臀部，而徒手反推原告，原告當場倒地，造成原告背部外傷及骨折,52,正當防衛，傷害故意，被告才是受害人,NaN,NaN,700000,50000
3,性騷擾,員工,1.954955,36.272727,1914876,NaN,被告趁原告尋求醫療協助之際，要求發生性行為，醫療行為不當，致原告腿部神經病變,52,術前已盡評估之事，原告為護士，於術前應已了解風險,趁病人不適且法逃脫醫師治療之依賴，違反道德,NaN,300000,250000
4,強制性交,待業,2.500000,36.272727,0,NaN,被告明知其為低智商而強制性交,52,二人互有情愫，並未暴力脅迫原告，經濟能力不佳，不應請求高額求償金,被告明知其為低智商而強制性交,NaN,1000000,400000


In [6]:
#犯罪類型 one hot encoding
crime_df = pd.get_dummies(df["犯罪類型"])
crime_df.head()

,乘機性交,強制性交,強制性交、猥褻,強制性交未遂,性騷擾,猥褻,與未成年人合意性交
0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0


In [7]:
#階級 one hot encoding
level_df = pd.get_dummies(df["階級"])
level_df.head()

,主管,員工,待業,老闆
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,0,1,0


In [8]:
# 財產種類 one hot encoding
t_investment = 0   # 投資
t_real_estate = 0  # 不動產
t_car = 0          # 汽車
t_house = 0        # 房屋
t_land = 0         # 土地
t_motorcycle = 0   # 機車

investment = []   # 投資
real_estate = []  # 不動產
car = []          # 汽車
house = []        # 房屋
land = []         # 土地
motorcycle = []   # 機車

for index in df["財產種類"]:
    if not pd.isnull(index):
        comma = index.split(",")
        for item in comma:
            temp = item.split(":")
            if temp[1] == "n":        # n設定為5
                temp[1] = 5
            
            if temp[0] == "投資":
                t_investment = temp[1]
            elif temp[0] == "不動產":
                t_real_estate = temp[1]
            elif temp[0] == "汽車":
                t_car = temp[1]
            elif temp[0] == "房屋":
                t_house = temp[1]
            elif temp[0] == "土地":
                t_land = temp[1]
            elif temp[0] == "機車":
                t_motorcycle = temp[1]
        investment.append(t_investment)
        real_estate.append(t_real_estate)
        car.append(t_car)
        house.append(t_house)
        land.append(t_land)
        motorcycle.append(t_motorcycle)
        
        t_investment = 0   # 投資
        t_real_estate = 0  # 不動產
        t_car = 0          # 汽車
        t_house = 0        # 房屋
        t_land = 0         # 土地
        t_motorcycle = 0   # 機車
    else:
        investment.append(0)
        real_estate.append(0)
        car.append(0)
        house.append(0)
        land.append(0)
        motorcycle.append(0)

c = {"投資":investment,
     "不動產":real_estate,
     "汽車":car,
     "房屋":house,
     "土地":land,
     "機車":motorcycle}
belongings = pd.DataFrame(c)
belongings.head()

,投資,不動產,汽車,房屋,土地,機車
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,1,1,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [9]:
# 刪除columns(犯罪類型、階級、財產種類、原告年紀)
df = df.drop(["犯罪類型", "階級", "財產種類", "原告年紀"], axis=1)
df.head()

,學歷,年齡,數額,原告主張,被告主張,有利原告,有利被告,原告要求金額,最後判賠金額
0,1.954955,36.272727,1000000,服用藥物昏沈不能抗拒,,趁原告服用安眠藥法抗拒，事後悔意從未道歉，指稱原告設局引誘被告,NaN,800000,500000
1,2.500000,36.272727,52540,因酒醉不知不能抗拒,當日因飲酒過量精神恍惚，證人之證詞互有矛盾，DNA未為23種型別比對，請求之賠償金額過高,被告利用原告酒醉不知不能抗拒,NaN,1300000,900000
2,1.954955,36.272727,36000,被告不滿原告以徒手拍打其臀部，而徒手反推原告，原告當場倒地，造成原告背部外傷及骨折,正當防衛，傷害故意，被告才是受害人,NaN,NaN,700000,50000
3,1.954955,36.272727,1914876,被告趁原告尋求醫療協助之際，要求發生性行為，醫療行為不當，致原告腿部神經病變,術前已盡評估之事，原告為護士，於術前應已了解風險,趁病人不適且法逃脫醫師治療之依賴，違反道德,NaN,300000,250000
4,2.500000,36.272727,0,被告明知其為低智商而強制性交,二人互有情愫，並未暴力脅迫原告，經濟能力不佳，不應請求高額求償金,被告明知其為低智商而強制性交,NaN,1000000,400000


# 語料庫前處理(斷詞)

In [10]:
# coding=utf-8
import jieba 
from pyhanlp import *
import re
import os
import gensim
analyer = PerceptronLexicalAnalyzer()
# load stopwords set
stopword_set = set()
with open('stopwords.txt','r', encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))

dff_hack = pd.read_csv('dff_badpeople.csv',encoding = 'utf_8')


fileTrainSeg=[]

col_name_list = ['原告主張','被告主張','有利原告','有利被告','主文']
def append_seq(col_name):    
    if col_name != '主文':
        print(col_name)
        for idx in range(len(df[col_name])):
            # 斷詞
    #         words = jieba.cut(df.iloc[idx][col_name],cut_all=False)
            article = df.iloc[idx][col_name]           
#             print(article)
            if type(article) == str:
                punctuation = """！？｡＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
                re_punctuation = "[{}] ".format(punctuation)
                article = re.sub(re_punctuation, "", article)
                words = analyer.segment(article)
            else:
                continue
            no_stopword_list = []
            for word in words:
#                 if (word not in stopword_set) and (not word.isdigit()) and (word.isalpha()):
                if (word.isalpha()):
                    no_stopword_list.append(word)   
            fileTrainSeg.append([' '.join(no_stopword_list)]) # 精確模式、同時也是預設模式
            # 因為會跑很久，檢核是否資料有持續在跑   
    elif col_name == '主文':
        print(col_name)
        for idx in range(len(dff_hack[col_name])):
            # 斷詞
    #         words = jieba.cut(df.iloc[idx][col_name],cut_all=False)
            article = dff_hack.iloc[idx][col_name]
            if type(article) == str:
                punctuation = """！？｡＂＃＄％＆＇（）＊＋－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏"""
                re_punctuation = r"[{}] ".format(punctuation)
                article = re.sub(re_punctuation, "", article)
                words = analyer.segment(article)
            else:
                continue
            words = analyer.segment(dff_hack.iloc[idx][col_name])
            no_stopword_list = []
            for word in words:
#                 if (word not in stopword_set) and (not word.isdigit()) and (word.isalpha()):
                if (word.isalpha()):                    
                    no_stopword_list.append(word)   
            fileTrainSeg.append([' '.join(no_stopword_list)]) # 精確模式、同時也是預設模式

In [11]:
# 若沒有則生成語料檔
if not os.path.exists('corpusSegDone.txt'):        
    for col in col_name_list:
        append_seq(col)
    # 將jieba的斷詞產出存檔
    fileSegWordDonePath ='corpusSegDone.txt'
    with open(fileSegWordDonePath,'wb') as fW:
        for i in range(len(fileTrainSeg)):
            fW.write(fileTrainSeg[i][0].encode('utf-8'))
            fW.write('\n'.encode('utf-8'))
else:
    print('corpusSegDone.txt Already exist!')

corpusSegDone.txt Already exist!


In [12]:
fileTrainSeg

[]

# 語料庫Word2Vec訓練

In [13]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from gensim.models import word2vec

# if not os.path.exists('corpusWord2Vec.bin'):  

# 將上述 jieba 斷詞後的 corpusSegDone.txt 轉成向量後存成 corpusWord2Vec.bin 檔：
sentences = gensim.models.word2vec.LineSentence('corpusSegDone.txt')
w2vec = gensim.models.Word2Vec(sentences, size=300, min_count=1,max_vocab_size=None)
    
w2vec.wv.save_word2vec_format('corpusWord2Vec.bin', binary=True)
w2vec = w2vec.wv.load_word2vec_format('corpusWord2Vec.bin', binary=True)
# model[fileTrainSeg[0][0].split()]
print(len(list(w2vec.vocab)))

# else:
#     print('corpusWord2Vec.bin Already exist!')

43267


In [14]:
list(w2vec.vocab)

# 原告主張
# 被告主張
# 有利原告
# 有利被告

['之',
 '原告',
 '被告',
 '為',
 '上訴',
 '人',
 '第',
 '於',
 '及',
 '其',
 '有',
 '與',
 '條',
 '上',
 '以',
 '行為',
 '等',
 '並',
 '項',
 '而',
 '判決',
 '請求',
 '應',
 '被',
 '在',
 '對',
 '本院',
 '女',
 '賠償',
 '或',
 '我',
 '所',
 '頁',
 '後',
 '的',
 '無',
 '元',
 '時',
 '號',
 '損害',
 '卷',
 '字',
 '依',
 '性交',
 '伊',
 '年度',
 '不',
 '因',
 '至',
 '即',
 '按',
 '一',
 '被害人',
 '經',
 '亦',
 '自',
 '開',
 '是',
 '係',
 '本件',
 '給付',
 '規定',
 '萬元',
 '理由',
 '見',
 '部分',
 '訴訟',
 '二',
 '且',
 '就',
 '執行',
 '刑事',
 '精神',
 '訴',
 '者',
 '駁回',
 '已',
 '又',
 '強制',
 '得',
 '侵權',
 '犯罪',
 '主張',
 '未',
 '日',
 '稱',
 '侵害',
 '前',
 '此',
 '假',
 '語',
 '均',
 '將',
 '財產',
 '如',
 '計算',
 'A',
 '兩造',
 '則',
 '由',
 '民事',
 '關係',
 'Ａ',
 '其他',
 '性',
 '內',
 '不法侵害',
 '中',
 '責任',
 '金',
 '可',
 '送達',
 '之身',
 '該',
 '男',
 '發生',
 '屬',
 '非',
 '他人',
 '他',
 '止',
 '慰撫',
 '甲',
 '相當',
 '說',
 '年',
 '民法',
 '事實',
 '起至',
 '爭',
 '清償',
 '歲',
 '金額',
 '系',
 '費用',
 '提起',
 '甲女',
 '臺灣',
 '雖',
 '下',
 '三',
 '侵',
 '本',
 '次',
 '之利',
 '息',
 '情形',
 '前段',
 '法院',
 '要',
 '你',
 '向',
 '負',
 '對於',
 '據',
 '遭',
 '明文'

# 製造Word2Vec詞庫

In [15]:
from gensim.models.keyedvectors import KeyedVectors
word_vectors = w2vec.wv.load_word2vec_format('corpusWord2Vec.bin', binary=True)


def make_w2v_embeddings(model, df, embedding_dim):  # 将词转化为词向量
    vocabs = {}  # 词序号
    vocabs_cnt = 0  # 词个数计数器

    vocabs_not_w2v = {}  # 无法用词向量表示的词
    vocabs_not_w2v_cnt = 0  # 无法用词向量表示的词个数计数器

    # 停用词
    # stops = set(open('stopwords.txt').read().strip().split('\n'))

    for index, row in df.iterrows():
        # 打印处理进度
        if index != 0 and index % 1000 == 0:
            print(str(index) + " sentences embedded.")

        for col in ['原告主張', '被告主張','有利原告','有利被告']:
            vecs = []  # vecs -> col to numbers representation
            words = row[col]
            try:
                for word in  words:
                    # if word in stops:  # 去停用词
                        # continue
                    if word not in list(model.vocab) and word not in vocabs_not_w2v:  # OOV的词放入不能用词向量表示的字典中，value为1
                        vocabs_not_w2v_cnt += 1
                        vocabs_not_w2v[word] = 1
                    if word not in vocabs:  # 非OOV词，提取出对应的id
                        vocabs_cnt += 1
                        vocabs[word] = vocabs_cnt
                        vecs.append(vocabs_cnt)
                    else:
                        vecs.append(vocabs[word])
                if len(vecs) == 0: vecs = [0] * 10
                df.at[index, col + '_vecs'] = vecs
            except Exception as e:
                df.at[index, col + '_vecs'] = [0] * 10

    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)  # 随机初始化一个形状为[全部词个数，词向量维度]的矩阵
    '''
    词1 [a1, a2, a3, ..., a60]
    词2 [b1, b2, b3, ..., b60]
    词3 [c1, c2, c3, ..., c60]
    '''
    embeddings[0] = 0  # 第一行用0填充，因为不存在index为0的词

    for index in vocabs:
        vocab_word = vocabs[index]
        if vocab_word in list(model.vocab):
            embeddings[index] = word_vectors[vocab_word]

    return df, embeddings

EMBEDDING_DIM = 300

for col in ['原告主張', '被告主張','有利原告','有利被告']:
    df[col + '_vecs'] = df[col]

df, embeddings = make_w2v_embeddings(word_vectors, df, embedding_dim=EMBEDDING_DIM)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


# 將原告主張,被告主張,有利原告,有利被告padding為一樣長度

In [16]:
max_len = 0
for i in df.原告主張_vecs:
    max_len = max(max_len, len(i))
for i in df.被告主張_vecs:
    max_len = max(max_len, len(i))
for i in df.有利原告_vecs:
    max_len = max(max_len, len(i))
for i in df.有利被告_vecs:
    max_len = max(max_len, len(i))
max_len

228

In [17]:
import itertools
from keras.preprocessing.sequence import pad_sequences

def split_and_zero_padding(df, max_seq_length):  # 调整tokens长度

    # 训练集矩阵转换成字典
    X = {'1': df['原告主張_vecs'], '2': df['被告主張_vecs'], '3': df['有利原告_vecs'], '4': df['有利被告_vecs']}

    # 调整到规定长度
    for dataset, side in itertools.product([X], ['1', '2','3','4']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

    return dataset



X_train = split_and_zero_padding(df,max_len)

hack_seq = X_train['1']  # 原告主張_vecs 輸出 Embedding編碼
hack_seq1 = X_train['2'] # 被告主張_vecs 輸出 Embedding編碼
hack_seq2 = X_train['3'] # 有利原告_vecs 輸出 Embedding編碼
hack_seq3 = X_train['4'] # 有利被告_vecs 輸出 Embedding編碼

Using TensorFlow backend.


In [18]:
print(hack_seq.shape)
print(hack_seq1.shape)
print(hack_seq2.shape)
print(hack_seq3.shape)

(210, 228)
(210, 228)
(210, 228)
(210, 228)


In [19]:
hack_seq[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  2,  3,
        4,  5,  6,  7,  8

# 刪除不必要的columns，concat所有dataframe

In [20]:
label = df["最後判賠金額"]
label = np.array(label, dtype='float64')
# 刪除columns(原告主張, 被告主張, 有利原告, 有利被告, 原告要求金額, 最後判賠金額)
df = df.drop(["原告主張", "被告主張", "有利原告", "有利被告", "原告要求金額", "最後判賠金額"], axis=1)

In [21]:
final_df = pd.concat([df,crime_df,level_df,belongings],axis=1)

In [22]:
final_df.head()

,學歷,年齡,數額,原告主張_vecs,被告主張_vecs,有利原告_vecs,有利被告_vecs,乘機性交,強制性交,強制性交、猥褻,...,主管,員工,待業,老闆,投資,不動產,汽車,房屋,土地,機車
0,1.954955,36.272727,1000000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[11, 12, 13, 1, 2, 14, 15, 3, 16, 9, 10, 17, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,2.500000,36.272727,52540,"[33, 34, 35, 7, 36, 7, 8, 9, 10]","[37, 38, 33, 39, 34, 40, 41, 42, 43, 44, 45, 1...","[32, 13, 72, 2, 12, 13, 34, 35, 7, 36, 7, 8, 9...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1.954955,36.272727,36000,"[32, 13, 7, 73, 12, 13, 74, 75, 76, 77, 78, 79...","[96, 37, 97, 98, 17, 92, 99, 100, 21, 17, 32, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,...,0,0,1,0,1,1,0,0,0,0
3,1.954955,36.272727,1914876,"[32, 13, 11, 12, 13, 104, 66, 105, 106, 107, 1...","[120, 121, 122, 123, 124, 125, 48, 18, 17, 12,...","[11, 118, 47, 7, 134, 135, 16, 136, 137, 105, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2.500000,36.272727,0,"[32, 13, 144, 36, 79, 57, 145, 146, 147, 82, 1...","[151, 47, 50, 51, 152, 153, 17, 154, 23, 155, ...","[32, 13, 144, 36, 79, 57, 145, 146, 147, 82, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,1,0,...,0,0,1,0,0,0,0,0,0,0


# 正規化

In [23]:
# from sklearn.preprocessing import MinMaxScaler
# import csv
# from sklearn import preprocessing

# #資料正規化
# def normalize(data):
# #     newdf= train_Aug.copy()
#     min_max_scaler = preprocessing.MinMaxScaler()                 
#     return min_max_scaler.fit_transform(data.reshape(-1,1))

# #資料正規化還原
# def denormalize(data, norm_value):
#     original_value = data.reshape(-1,1)
#     norm_value = norm_value.reshape(-1,1)
    
#     min_max_scaler = preprocessing.MinMaxScaler()
#     min_max_scaler.fit_transform(original_value)
#     denorm_value = min_max_scaler.inverse_transform(norm_value)        
#     return denorm_value

# # train_y = normalize(train_y)

# 特徵

In [24]:
# 數值
# num_cols = list(final_df.columns[:20])  # num_cols

num_cols = ['學歷', '年齡', '數額','投資', '不動產', '汽車', '房屋', '土地', '機車']
num_df = final_df[num_cols]
# num_df = num_df.drop(['強制性交','猥褻','強制性交未遂','性騷擾','猥褻','與未成年人合意性交','乘機性交','強制性交、猥褻'], axis=1)
# num_df = num_df.drop(['原告主張_vecs','被告主張_vecs','有利原告_vecs','有利被告_vecs'], axis=1)
num_cols = num_df.columns
num_data = np.array(num_df)
num_df.head()

,學歷,年齡,數額,投資,不動產,汽車,房屋,土地,機車
0,1.954955,36.272727,1000000,0,0,0,0,0,0
1,2.500000,36.272727,52540,0,0,0,0,0,0
2,1.954955,36.272727,36000,1,1,0,0,0,0
3,1.954955,36.272727,1914876,0,0,0,0,0,0
4,2.500000,36.272727,0,0,0,0,0,0,0


In [25]:
# 類別
from sklearn.preprocessing import LabelEncoder
cat_cols = ['主管','員工', '待業', '老闆','乘機性交', '強制性交', '強制性交、猥褻', '強制性交未遂', '性騷擾', '猥褻', '與未成年人合意性交']
cat_df = final_df[cat_cols]
cat_data = np.array(cat_df)
cat_df.head()

,主管,員工,待業,老闆,乘機性交,強制性交,強制性交、猥褻,強制性交未遂,性騷擾,猥褻,與未成年人合意性交
0,0,0,0,1,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,1,0,0,0,0,0


In [26]:
hack_seq_mat = pd.DataFrame(hack_seq) # setence_cols
hack_seq_mat1 = pd.DataFrame(hack_seq1)
hack_seq_mat2 = pd.DataFrame(hack_seq2)
hack_seq_mat3 = pd.DataFrame(hack_seq3)
hack_seq_mat.shape

(210, 228)

In [27]:
from sklearn.model_selection import train_test_split

# data split
train_num_X,test_num_X, \
train_cat_X,test_cat_X, \
train_hack_seq_X, test_hack_seq_X, \
train_hack_seq1_X, test_hack_seq1_X, \
train_hack_seq2_X, test_hack_seq2_X, \
train_hack_seq3_X, test_hack_seq3_X, \
train_y,test_y = train_test_split(num_data,cat_data,hack_seq,hack_seq1,hack_seq2,hack_seq3,label, test_size=0.10, random_state=0)

# 模型訓練(lgbm)

In [28]:
import lightgbm as lgb

# Create the LightGBM data containers
train_data = lgb.Dataset(data=train_num_X,label=train_y)
# train_data1 = lgb.Dataset(data=train_cat_X,label=train_y)

test_data = lgb.Dataset(data=test_num_X,label=test_y)
# test_data1 = lgb.Dataset(data=test_cat_X,label=test_y)

In [29]:
param = {'boosting_type':'gbdt',
         'objective':'regression',
         
         'num_leaves':7, 
         'max_depth':4,
         'learning_rate':0.01,
         
         'metric': 'rmse',
         'subsample':1.0,
         'colsample_bytree':1.0,
         'n_estimators':100,
         'bagging_fraction': 1.0,
         'feature_fraction': 0.8,
         'min_child_samples':13,
         'reg_lambda': 0.08,
         'reg_alpha': 0
         
        }

model = lgb.train(param,train_data,valid_sets=test_data)

[1]	valid_0's rmse: 364674
[2]	valid_0's rmse: 366417
[3]	valid_0's rmse: 366461
[4]	valid_0's rmse: 368214
[5]	valid_0's rmse: 369981
[6]	valid_0's rmse: 371686
[7]	valid_0's rmse: 373404
[8]	valid_0's rmse: 375203
[9]	valid_0's rmse: 375406
[10]	valid_0's rmse: 377138
[11]	valid_0's rmse: 377242
[12]	valid_0's rmse: 377442
[13]	valid_0's rmse: 379172
[14]	valid_0's rmse: 380910
[15]	valid_0's rmse: 382656
[16]	valid_0's rmse: 384470
[17]	valid_0's rmse: 386228
[18]	valid_0's rmse: 388049
[19]	valid_0's rmse: 389814
[20]	valid_0's rmse: 391583
[21]	valid_0's rmse: 391553
[22]	valid_0's rmse: 391527
[23]	valid_0's rmse: 393289
[24]	valid_0's rmse: 395054
[25]	valid_0's rmse: 395133
[26]	valid_0's rmse: 395303
[27]	valid_0's rmse: 395474
[28]	valid_0's rmse: 397276
[29]	valid_0's rmse: 399025
[30]	valid_0's rmse: 400775
[31]	valid_0's rmse: 402525
[32]	valid_0's rmse: 404321
[33]	valid_0's rmse: 406068
[34]	valid_0's rmse: 406223
[35]	valid_0's rmse: 408008
[36]	valid_0's rmse: 408072
[

D:\Anaconda\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [30]:
Y_pred = model.predict(test_num_X)
Y_pred

array([ 533137.57795126, 1010660.78912409, 1024175.36946262,
        586644.50444467, 1029286.80964965,  616552.98568253,
        488556.62099755,  597352.79330185,  521575.8044408 ,
        623326.90686583,  643260.28202664, 1010660.78912409,
        575892.5313131 ,  977995.2906531 ,  577421.79317403,
        520839.11339786,  640125.7957627 ,  557707.65215734,
        583034.58004731,  616552.98568253,  566760.39714829])

In [31]:
test_y

array([ 900000.,  400000.,  600000.,  300000.,  250000.,  150000.,
       1200000.,  300000.,  800000., 1500000., 1000000.,  400000.,
        800000.,  250000.,  800000.,  700000.,  500000., 1000000.,
       1200000.,  500000.,  400000.])

In [32]:
error = []
for i in range(len(test_y)):
    error.append(test_y[i] - Y_pred[i])
    
#print("Errors: ", error)
#print(error)
squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)#平方
    absError.append(abs(val))#誤差絕對值
    
#print("Square Error: ", squaredError)
#print("Absolute Value of Error: ", absError)
print("MSE = ", sum(squaredError) / len(squaredError))#平均平方誤差MSE


#from math import sqrt
#print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#平均平方根誤差RMSE
print("MAE = ", sum(absError) / len(absError))#平均絕對誤差MAE

MSE =  230214224180.87534
MAE =  423905.507031638
